## 写成RV汇编代码
```
Loop: 
        fld   f0, 0(x1)
        fadd.d  f4, f0, f2
        fsd   f4, 0(x1)
        addi   x1, x1, 8
        bne   x1, x2, Loop
```

In [8]:
from prettytable import PrettyTable

class Delay_analysis:
    def __init__(self, instructions):
        self.instructions = instructions
        self.table = []
        self.latency = [[0 for _ in range(len(instructions) + 1)] for _ in range(len(instructions) + 1)]

    def is_fp_alu_op(self, instr):
        if instr[0][0] == "f":
            op = instr[0][1:]
            if op == "add.d":
                return True
        return False

    def is_store_double(self, instr):
        if instr[0][0] == "f":
            op = instr[0][1:]
            if op == "sd":
                return True
        return False

    def is_load_double(self, instr):
        if instr[0][0] == "f":
            op = instr[0][1:]
            if op == "ld":
                return True
        return False

    def is_integer_op(self, instr):
        if instr[0][0] != "f":
            op = instr[0]
            if op == "add" or op == "addi":
                return True
        return False

    def display_table(self):
        tb = PrettyTable(["cycle", "instr"])
        for i in range(len(self.table)):
            tb.add_row([self.table[i][0], self.table[i][1]])
        print(tb)
        
    def calc_latency(self, instr1, instr2):
        if self.is_fp_alu_op(instr1):
            if self.is_fp_alu_op(instr2):
                return 3
            elif self.is_store_double(instr2):
                return 2
        elif self.is_load_double(instr1):
            if self.is_fp_alu_op(instr2):
                return 1
        elif self.is_integer_op(instr1):
            if instr2[0] == "bne":
                if instr1[1] == instr2[1] or instr1[1] == instr2[2]:
                    return 1
        return 0
            
    def analysis(self):
        for i in range(len(self.instructions)):
            instr1 = self.instructions[i]
            if instr1[0] == "bne" and i == (len(self.instructions) - 1):
                self.latency[i][i + 1] = 1
            for j in range(i + 1, len(self.instructions)):
                instr2 = self.instructions[j]
                self.latency[i][j] = self.calc_latency(instr1, instr2)

        for i in range(len(self.instructions)):
            distance = 0
            for j in range(i + 1, len(self.instructions)):
                distance += 1 + self.latency[j - 1][j]
                if distance <= self.latency[i][j]:
                    self.latency[i][i + 1] += self.latency[i][j] - distance + 1
                    distance += self.latency[i][j] - distance + 1

        cycle = 0
        for i in range(len(self.instructions)):
            instr = self.instructions[i]
            cycle += 1
            self.table.append((cycle, instr))
            for j in range(self.latency[i][i + 1]):
                cycle += 1
                self.table.append((cycle, "stall"))

        return self


instructions_unscheduled = [
    ["fld", "f0", "0", "x1"],
    ["fadd.d", "f4", "f0", "f2"],
    ["fsd", "f4", "0", "x1"],
    ["addi", "x1", "x1", "8"],
    ["bne", "x1", "x2", "Loop"],
]

print("Unscheduled:")
Delay_analysis(instructions_unscheduled).analysis().display_table()

Unscheduled:
+-------+------------------------------+
| cycle |            instr             |
+-------+------------------------------+
|   1   |   ['fld', 'f0', '0', 'x1']   |
|   2   |            stall             |
|   3   | ['fadd.d', 'f4', 'f0', 'f2'] |
|   4   |            stall             |
|   5   |            stall             |
|   6   |   ['fsd', 'f4', '0', 'x1']   |
|   7   |  ['addi', 'x1', 'x1', '8']   |
|   8   |            stall             |
|   9   | ['bne', 'x1', 'x2', 'Loop']  |
|   10  |            stall             |
+-------+------------------------------+


## 调度后
```
Loop: 
        fld   f0, 0(x1)
        addi   x1, x1, 8
        fadd.d  f4, f0, f2
        bne   x1, x2, Loop
        fsd   f4, 0(x1)
```

In [9]:
instructions_scheduled = [
    ["fld", "f0", "0", "x1"],
    ["addi", "x1", "x1", "8"],
    ["fadd.d", "f4", "f0", "f2"],
    ["bne", "x1", "x2", "Loop"],
    ["fsd", "f4", "0", "x1"],
]

print("Scheduled:")
Delay_analysis(instructions_scheduled).analysis().display_table()

Scheduled:
+-------+------------------------------+
| cycle |            instr             |
+-------+------------------------------+
|   1   |   ['fld', 'f0', '0', 'x1']   |
|   2   |  ['addi', 'x1', 'x1', '8']   |
|   3   | ['fadd.d', 'f4', 'f0', 'f2'] |
|   4   |            stall             |
|   5   | ['bne', 'x1', 'x2', 'Loop']  |
|   6   |   ['fsd', 'f4', '0', 'x1']   |
+-------+------------------------------+
